# Jupyter Notebook to obtain VLASS cutouts 
## By G. R. Sivakoff (University of Alberta); B. Sebastian, (University of Manitoba)

The provided Python code automates the process of retrieving URLs for VLASS quick look image fits files for 3' X 3' boxes centred on a set of sky coordinates. The code reads a catalog with coordinates from a CSV file, each represented by columns RA and DEC. For each coordinate, it fetches the corresponding VLASS image URL through the <b><em>getCadcVlassUrl()</em></b> function. These URLs are stored in a new column within the dataset. The augmented data, now including URLs for VLASS images, is finally saved back to a CSV file for further use and analysis.

* <font color=maroon><b> Specify the catalog input. </b></font><br>
Download the csv version of an entire catalog (e.g.,
<em>CIRADA_VLASS1QLv3.1_table1_components.csv</em>, <em>CIRADA_VLASS2QLv2_table1_components.csv</em>, or
<em>CIRADA_VLASS2SEv2_table1_components.csv</em>) and make a subset of the entire catalog. 
Specify the location of the catalog in Cell 2 of this notebook. Click [here](#my_target_cell) to navigate to the Cell 2.
This code assumes the python code is being run in the same directory as the downloaded file.
 
* <font color=maroon><b>Input the correct epoch and the imagetype.</b></font><br>
In this notebook, we extract URLs for the 'Epoch 1 Quick Look' images. Alternative options are commented next to the parameters in [Cell 2](#my_target_cell).
* <font color=maroon><b>Comment out "testRunFlag = True" in Cell 2 if you want to retrieve the CADC urls for the entire catalog.</b></font><br>
If the variable <em>testRunFlag</em> is "True", only the first five rows will be affected; if the variable is "False", the code operates on the entire catalog. This variable is extremely useful for testing the code before running on a large CSV file.   
* <font color=maroon><b>This code does not produce URLs for mosaiced images.</b></font><br>
In the case that there are multiple cutout images appropriate for a source in your query, only the final URL from the list of URLS for that source will be returned. This URL should be for the latest (by observation date) image.
* <font color=maroon><b>Some tests of this took 5 seconds per source! Check back to see if we develop speedier methods.</b></font><br>

## Note:

The code snippets here are based off code that are helping add VLASS Quicklook RMS Functionality and Single Epoch Functionality to the CIRADA cutout service. These are only meant to be examples and they may need to be written with more proper coding principles, upgraded for working with Python versions > 3.7, have more extensive unit testing performed, and undergo optimization refactoring.

Nevertheless, some users may find these useful if they do not want to use a Dockerized version of the CIRADA command-line cutout service or the online CIRADA cutout service. Please note that CIRADA is also looking to implement an API for the online service that would include full access to all surveys in the CIRADA cutout services.

## Verified Python Libary Versions

Please note that this code may work with other versions of python and the libraries listed below; however, only the below versions have been tested have not been tested.
* Python 3.7, 3.8, 3.9, 3.10
* numpy 1.23.5
* pandas 1.5.3
* astropy 5.3.1
* astroquery-0.4.6

## Code Setup

Please use this cell to edit important variables for this notebook.

* The variable <em>onColab</em> ensures astroquery is installed when Google Colab runs this code.
* The variable <em>testRunFlag</em> allows the code to only operate on the first 5 sources in a catalog for testing purposes.
* The variable <em>radius</em> sets the size of the square image to be extracted.
* The array <em>epoch</em> sets the epochs to be considered, although onbly the latest epoch for a source will be output.
* The array <em>imageType</em> sets the type of image to be output. Please note this example code that outputs to a single column per source for a catalog works correctly only when one <em>imageType</em> is specified.

In [1]:
onColab = False
onColab = True # Uncomment only if running on Google Colab



from pathlib import Path # Part of python

import numpy as np
import pandas as pd

from astropy import units as u
from astropy.coordinates import SkyCoord

if onColab:
  from os import system # Part of python
  cmd = "pip install astroquery"
  system(cmd)

from astroquery.cadc import Cadc


radius = 3.0*u.arcmin/2. # Default to a 3' x 3' square cutout

# Note, only the newest (by observation date) cutout is returned


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


<a id='my_target_cell'></a>
## Provide catalog inputs here

In [2]:
cat='outsse.csv' # provide the path to your catalog
epoch=np.array(['1.1','1.2']) # Potential Epochs are '1.1','1.2','2.1','2.2','3.1','3.2'

imageType=np.array(['ql.tt0']) #Note: potential imagetypes are 'ql.tt0','ql.tt0.rms','se.tt0','se.tt0.rms','se.tt1','se.tt1.rms','se.alpha.error','se.alpha'

testRunFlag = False # Run on full catalog
testRunFlag = True  # Run on first five lines of catalog


## Core Code Backend Functions

The following core code backend functions enable queries by coordinates. Dealing with catalogs will be done in a different cell.

In [3]:
def getVlassFitsNames(cadc,cadcResults):
  import numpy as np
  # Given a CADC astroquery connection and the return of an already
  # executed base query, get the names and set the types of all FITS files
  # and classify their type within VLASS
  # 1. Search all auxillary files for an observation for fits files
  # 2. Classify FITS file type
  fitsNames = np.array([])
  vlassTypes = np.array([])
  urls = cadc.get_data_urls(cadcResults,include_auxiliaries=True)
  for url in np.sort(urls):
    if (url[-4:] =='fits'):
      tempFitsName = url.rsplit('VLASS/')[1]
      fitsNames = np.append(fitsNames,tempFitsName)
      vlassTypes = np.append(vlassTypes,classifyVlassFits(tempFitsName))
  return (fitsNames, vlassTypes)

def classifyVlassFits(fitsName):
  # Given a VLASS FITS file following standard naming conventions,
  # determine the type of image the FITS file is.
  # All VLASS FITS files except spectral index (alpha) maps should follow
  # *pbcor.AAA.subim.fits, where AAA can be a variable length string that
  # indicates the file type. VLASS FITS files that are spectral index
  # (alpha) maps shoudl follow *alpha.AAA.subim.fits.
  # There does not seem to be a simple CAOM query that can do this, which
  # is why we are using the name.
  if ("pbcor" in fitsName):
    suffix = fitsName.split('pbcor.')
  elif ("alpha" in fitsName):
    suffix = fitsName.split('alpha.')
    suffix[1] = "alpha."+suffix[1]
  if len(suffix) > 1:
    suffix = suffix[1]
  else:
    return(None)
  if len(suffix)>13:
    vlassType=suffix[:-11]
  else:
    vlassType=None
  return(vlassType)


def getCoordVlass(targetCoordinate,targetRadius,
                  targetEpochs,targetTypes,
                  cadc):
  # Given a target coordinate, radius, Epochs, and image types, this
  # uses an active CADC astroquery connection that has already been
  # verified to have results to return the URL for VLASS cutout download
  # and the image type among VLASS options.

  from astroquery.cadc import Cadc
  from astropy import units as u
  from astropy import table
  from urllib import parse
  import numpy as np


  fitsNames         = np.array([])
  vlassTypes        = np.array([])
  CadcUrl           = np.array([])
  completedSubtiles = np.array([])

  # Note that the following warnings (with path changes based on python
  # installation) show up everytime this function is run.
  # WARNING: UnitsWarning: The unit 'pix' has been deprecated in the
  #   VOUnit standard. [astropy.units.format.utils]
  # WARNING:astroquery:UnitsWarning: The unit 'pix' has been deprecated in
  #   the VOUnit standard.
  # /usr/local/lib/python3.7/dist-packages/astropy/table/table.py:3401:
  #   FutureWarning: elementwise == comparison failed and returning scalar
  #   instead; this will raise an error or perform elementwise comparison
  #   in
  #   the future.
  #     result = (self.as_array().data == other) &
  #        (self.mask == false_mask)
  # /usr/local/lib/python3.7/dist-packages/astropy/table/table.py:3409:
  #   FutureWarning: elementwise == comparison failed and returning scalar
  #   instead; this will raise an error or perform elementwise comparison
  #   in
  results = cadc.query_region(targetCoordinate, targetRadius,
                              collection='VLASS')
  if len(results) == 0:
    print("No Valid Coordinate")
    return(None)
  else:
    # Reject QA failed observations
    passedResults =   results[results['requirements_flag']!='fail']
    # Sort by obstime
    sortedSubtiles = (passedResults.group_by('time_bounds_lower'))
    for subtile in sortedSubtiles:
      # This needs to be done becausethe same observationURI can
      # have multiple results. In particular, one for Quick Look
      # (currently calibrationLevel==2) and one for Single Epoch
      # (currently calibrationLevel==3). This is not ideal, but, ...
      if subtile['observationURI'] not in completedSubtiles:
        completedSubtiles = np.append(completedSubtiles,
                                      subtile['observationURI'])
        # Gets the epoch of the current subtile result
        observedEpoch = subtile['proposal_id'][5:]
        # Gets the observationURI of the current subtile result
        observationURI = subtile['observationURI']
        # Only return urls for targetted Epochs
        if(observedEpoch in targetEpochs):
          # Since I think QL should always be there [we should check this
          # with NRAO], get all FITS files associated with an
          # observationURI by first only considering the QL subtile
          # results. This is done so one does not get the same files
          # twice, once for the astroquery result for QL and one for the
          # astroquery result for SE.
          resultSelection = np.logical_and(
              results['observationURI']==observationURI,
              results['calibrationLevel']==2)
          subtileResult = results[resultSelection]
          # Gets the VLASS FITS names for a QL observationURI
          (tempFitsNames,tempVlassTypes) = getVlassFitsNames(cadc,
                                                            subtileResult)
          # This is not an ideal for loop. It should probably be converted
          # to np.where
          # Ideally CADC should only have one set of QL and one set of SE
          # images per ObservationURI. There is no real error checking
          # being done about this...
          for ii in np.arange(len(tempVlassTypes)):
            if(tempVlassTypes[ii]=="tt0"):
              if("ql" in tempFitsNames[ii]):
                baseUrl = cadc.get_image_list(subtileResult,
                                              targetCoordinate,
                                              targetRadius)
                baseUrl = baseUrl[0]
                qlName = tempFitsNames[ii]
          # Now that we have the Quick Look default result, consider
          # all results with the same observationURI
          resultSelection = results['observationURI']==observationURI,
          subtileResult = results[resultSelection]
          # Gets the VLASS FITS names for a QL or SE observationURI
          (tempFitsNames,tempVlassTypes) = getVlassFitsNames(cadc,
                                                            subtileResult)
          for ii in np.arange(len(tempVlassTypes)):
            # Added July 17, 2023 to fix a bug
            url = np.char.replace(baseUrl,
                                  np.char.replace(qlName,
                                           "+","%2B"),
                                  np.char.replace(tempFitsNames[ii],
                                                  "+","%2B"))
            # This probably should use some better error catching.
            tempVlassType = "NOSUCHTYPE"
            # Prepend SE versus QL
            if("se" in tempFitsNames[ii]):
                tempVlassType = "se."+tempVlassTypes[ii]
            elif ("ql" in tempFitsNames[ii]):
                tempVlassType = "ql."+tempVlassTypes[ii]
            # Append values of CADC URL, FITS file names (not currently
            #used),
            # and VLass Image Types
            if (tempVlassType in targetTypes):
              CadcUrl    = np.append(CadcUrl,url)
              fitsNames  = np.append(fitsNames,tempFitsNames[ii])
              vlassTypes = np.append(vlassTypes,tempVlassType)
    return(CadcUrl,vlassTypes)

def getCadcVlassUrl(coordinate, radius,
                    epoch=np.array(["1.1","1.2",
                                    "2.1","2.2",
                                    "3.1","3.2"]),
                    imageType = np.array(['ql.tt0',
                                          'ql.tt0.rms',
                                          'se.tt0',
                                          'se.tt0.rms',
                                          'se.tt1',
                                          'se.tt1.rms',
                                          'se.alpha.error',
                                          'se.alpha'])):
  # Given a target coordinate, radius, Epochs, and image types, this
  # uses an active CADC astroquery connection to first verify that there
  # will be results, and then to return the URL for VLASS cutout download
  # and the image type among VLASS options.
  from astroquery.cadc import Cadc
  from astropy import units as u

  cadc = Cadc()
  inVlassCheck = checkCoordVlass(coordinate,epoch,cadc)
  if True in inVlassCheck:
    return(getCoordVlass(coordinate,radius,epoch,imageType,cadc))
  return(None,None)

def checkCoordVlass(targetCoordinate,
                    targetEpochs,cadc):
  # Given a target coordinate and Epochs, this
  # uses an active CADC astroquery connection to verify that there
  # will be results.

  import numpy as np
  from astroquery.cadc import Cadc
  from astropy import units as u

  checkPassed =  np.array([], dtype='bool')

  # 2 arcsecond availability for quick check
  existenceRadius = 2/3600.*u.deg
  # Note that the following warnings (with path changes based on python
  # installation) show up everytime this function is run.
  # WARNING: UnitsWarning: The unit 'pix' has been deprecated in the
  #   VOUnit standard. [astropy.units.format.utils]
  # WARNING:astroquery:UnitsWarning: The unit 'pix' has been deprecated in
  #   the VOUnit standard.
  # /usr/local/lib/python3.7/dist-packages/astropy/table/table.py:3401:
  #   FutureWarning: elementwise == comparison failed and returning scalar
  #   instead; this will raise an error or perform elementwise comparison
  #   in
  #   the future.
  #     result = (self.as_array().data == other) &
  #        (self.mask == false_mask)
  # /usr/local/lib/python3.7/dist-packages/astropy/table/table.py:3409:
  #   FutureWarning: elementwise == comparison failed and returning scalar
  #   instead; this will raise an error or perform elementwise comparison
  #   in
  #   the future. result = self.as_array() == other
  results = cadc.query_region(targetCoordinate, existenceRadius,
                              collection='VLASS', verbose=False)
  if len(results) == 0:
    print("No Valid Coordinate")
    checkPassed = np.append(checkPassed,False)
  else:
    # Sort by obstime
    sortedSubtiles = (results.group_by('time_bounds_lower'))
    for subtile in sortedSubtiles:
      observedEpoch = subtile['proposal_id'][5:]
      if(observedEpoch in targetEpochs):
        checkPassed = np.append(checkPassed,True)
      else:
        checkPassed = np.append(checkPassed,False)
  return(checkPassed)

## Execute Code

The following cell allow users to specify the catalog they wish to add URLs too. The cell will execute the code to get new URLS and append it to a new catalog file.

In [4]:
catalogFileExists = False

while not(catalogFileExists):
  
  catPath = Path(cat)

  if catPath.is_file():
    catalogFileExists = True
    print('\nReading in catalog file ({}):'.format(cat))
    catalog=pd.read_csv(cat)

    # Runs code only on first five lines when testing
    if testRunFlag:
      catalog=catalog.head(5)

    # Create a new empty column for the URLs
    catalog['URL_cutout'] = ""

    for index, row in catalog.iterrows():
      coordinate = SkyCoord(f'{row["RA"]} {row["DEC"]}', unit='deg')
      (CadcUrl, vlassTypes) = getCadcVlassUrl(coordinate,
                                              radius,
                                              epoch=epoch,
                                              imageType=imageType)
      # Save the URL in the new column
      if type(CadcUrl) == type(None):
        catalog.at[index, 'URL_cutout'] = None
      elif len(CadcUrl) > 1:
        catalog.at[index, 'URL_cutout'] = CadcUrl[-1]

    # Write the DataFrame with the new column back to a CSV file
    newCat = 'output_catalog.csv'
    print('\nOutputting new catalog file ({}):'.format(newCat))
    catalog.to_csv(newCat, index=False)

  else:
    print('\nYour catalog file "{}"'.format(cat)+
             'does not appear to exist\n'+
          'Please specify a valid path when rerunning this cell.\n')


Reading in catalog file (outsse.csv):


/tmp/ipykernel_48649/2721378606.py:95: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if subtile['observationURI'] not in completedSubtiles:
/home/biny/.local/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(



Outputting new catalog file (output_catalog.csv):
